# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium 

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sallisaw,35.4604,-94.7875,36.08,52,0,5.66,US,1693938475
1,1,luderitz,-26.6481,15.1594,13.88,53,87,13.53,NaN,1693938475
2,2,waitangi,-43.9535,-176.5597,10.46,94,100,0.45,NZ,1693938476
3,3,lanxi,29.2195,119.4770,24.25,96,100,1.41,CN,1693938476
4,4,puerto natales,-51.7236,-72.4875,4.91,46,44,1.08,CL,1693938476


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
coordinates = {"Lat" : (city_data_df["Lat"]),
               "Lng" : (city_data_df["Lng"]),
               "City" : (city_data_df["City"]),
               "Humidity" : (city_data_df["Humidity"]) 
               }
coordinates_df = pd.DataFrame(coordinates)
coordinates_df

,Lat,Lng,City,Humidity
0,35.4604,-94.7875,sallisaw,52
1,-26.6481,15.1594,luderitz,53
2,-43.9535,-176.5597,waitangi,94
3,29.2195,119.4770,lanxi,96
4,-51.7236,-72.4875,puerto natales,46
...,...,...,...,...
593,36.6167,101.7667,xining,77
594,56.7268,-111.3810,fort mcmurray,36
595,35.1390,-79.0060,fort bragg,46
596,-38.6533,178.0042,gisborne,86


In [4]:
%%capture --no-display

# Display the map
humidity_map = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)
humidity_map




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df['Max Temp'] >= 10) &
                                 (city_data_df['Max Temp'] < 30).dropna()]

# Display sample data
display(ideal_weather)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,luderitz,-26.6481,15.1594,13.88,53,87,13.53,NaN,1693938475
2,2,waitangi,-43.9535,-176.5597,10.46,94,100,0.45,NZ,1693938476
3,3,lanxi,29.2195,119.4770,24.25,96,100,1.41,CN,1693938476
7,7,gharyan,32.1722,13.0203,20.52,57,13,7.26,LY,1693938477
11,11,montecristi,-1.0500,-80.6667,27.90,65,100,3.60,EC,1693938477
...,...,...,...,...,...,...,...,...,...,...
590,590,matsue,35.4722,133.0506,24.00,100,75,2.57,JP,1693938657
592,592,carutapera,-1.1950,-46.0200,29.64,63,35,5.27,BR,1693938658
593,593,xining,36.6167,101.7667,15.81,77,47,1.12,CN,1693938658
594,594,fort mcmurray,56.7268,-111.3810,15.77,36,0,5.14,CA,1693938658


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,luderitz,NaN,-26.6481,15.1594,53,
2,waitangi,NZ,-43.9535,-176.5597,94,
3,lanxi,CN,29.2195,119.4770,96,
7,gharyan,LY,32.1722,13.0203,57,
11,montecristi,EC,-1.0500,-80.6667,65,
...,...,...,...,...,...,...
590,matsue,JP,35.4722,133.0506,100,
592,carutapera,BR,-1.1950,-46.0200,63,
593,xining,CN,36.6167,101.7667,77,
594,fort mcmurray,CA,56.7268,-111.3810,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {
    "categories" : categories,
    "apikey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity : {longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
luderitz - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
lanxi - nearest hotel: No hotel found
gharyan - nearest hotel: No hotel found
montecristi - nearest hotel: No hotel found
daru - nearest hotel: No hotel found
tostado - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
upington - nearest hotel: No hotel found
banikoara - nearest hotel: No hotel found
muang xay - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
ullal - nearest hotel: No hotel found
taltal - nearest hotel: No hotel found
qacha's nek - nearest hotel: No hotel found
yunyang - nearest hotel: No hotel found
shira - nearest hotel: No hotel found
chinde - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
nemuro - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
1,luderitz,NaN,-26.6481,15.1594,53,No hotel found
2,waitangi,NZ,-43.9535,-176.5597,94,No hotel found
3,lanxi,CN,29.2195,119.4770,96,No hotel found
7,gharyan,LY,32.1722,13.0203,57,No hotel found
11,montecristi,EC,-1.0500,-80.6667,65,No hotel found
...,...,...,...,...,...,...
590,matsue,JP,35.4722,133.0506,100,No hotel found
592,carutapera,BR,-1.1950,-46.0200,63,No hotel found
593,xining,CN,36.6167,101.7667,77,No hotel found
594,fort mcmurray,CA,56.7268,-111.3810,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]  
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)